# Importing Libraries

In [ ]:
# Data Analysis
import pandas as pd
import numpy as np
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt
import pickle as pkl
from scipy import sparse

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Text Processing
import re
import itertools
import string
import collections
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Machine Learning packages
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import sklearn.cluster as cluster
from sklearn.manifold import TSNE

# Model training and evaluation
from sklearn.model_selection import train_test_split

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance

#Metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, multilabel_confusion_matrix, confusion_matrix
from sklearn.metrics import classification_report

# Ignore noise warning
import warnings
warnings.filterwarnings("ignore")

## Loading Dataset

In [ ]:
data_set_personality = pd.read_csv("mbti_1.csv")

### printing the top tuples

In [ ]:
data_set_personality.head()

### Describing the data set

In [ ]:
data_set_personality.describe()

In [ ]:
# printing the number of columns and rows
data_set_personality.shape

## Data Cleaning

### Dropping duplicates values if any are present

In [ ]:
data_set_personality.drop_duplicates

In [ ]:
# after dropping duplicates checking for the number of rows
print(data_set_personality)

# Checking for null values

In [ ]:
null_values = data_set_personality.isnull().sum()
null_values

In [ ]:
# checking the dataset information
data_set_personality.dtypes

No null values present in the dataset and only 2 columns and 8675 rows present in the data_set_personality.
The values are textual therefore has to be converted to numerical form to train the ML model

# Data Information

In [ ]:
data_set_personality.dtypes

In [ ]:
data_set_personality.info()

In [ ]:
types = np.unique(np.array(data_set_personality['type']))
types

In [ ]:
#Group by allows you to split your data into separate groups to perform computations for better analysis.
total = data_set_personality.groupby(['type']).count()*50
total

# Data Visualization

In [ ]:
plt.figure(figsize = (12,4))
plt.bar(np.array(total.index), height = total['posts'],)
plt.xlabel('Personality types', size = 14)
plt.ylabel('No. of posts available', size = 14)
plt.title('Total posts for each personality type')

The dataset is clearly unbalanced throughout the different classes. We observe that some of the personality types has a lot more data than others, the most common Kaggle users personality is INFP (Introvert Intuition Feeling Perceiving).

In [ ]:
#Plotting this in descending order for better understanding of this visualization
cnt_srs = data_set_personality['type'].value_counts()
plt.figure(figsize=(12,4))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8)
plt.xlabel('Personality types', fontsize=12)
plt.ylabel('No. of posts availables', fontsize=12)
plt.show()

### SWARM PLOT

In [ ]:
df = data_set_personality.copy()
#this function counts the no of words in each post of a user
def var_row(row):
    l = []
    for i in row.split('|||'):
        l.append(len(i.split()))
    return np.var(l)

#this function counts the no of words per post out of the total 50 posts in the whole row
df['words_per_comment'] = df['posts'].apply(lambda x: len(x.split())/50)
df['variance_of_word_counts'] = df['posts'].apply(lambda x: var_row(x))

plt.figure(figsize=(15,10))
sns.swarmplot("type", "words_per_comment", data=df)

### Joint Plot

In [ ]:
plt.figure(figsize=(15,10))
sns.jointplot("variance_of_word_counts", "words_per_comment", data=df, kind="hex")

### Joint Plots for each Personality Type

In [ ]:
def plot_jointplot(mbti_type, axs, titles):
    df_1 = df[df['type'] == mbti_type]
    sns.jointplot("variance_of_word_counts", "words_per_comment", data=df_1, kind="hex", ax = axs, title = titles)

plt.figure(figsize=(24, 5))
i = df['type'].unique()
k = 0

for m in range(1,3):
    for n in range(1,7):
        df_1 = df[df['type'] == i[k]]
        sns.jointplot("variance_of_word_counts", "words_per_comment", data=df_1, kind="hex" )
        plt.title(i[k])
        k+=1
plt.show()

* For all the plots you can see that most of the posts have words btw 100-150 and most of no. of words per comment by a user is nearly 25 to 30 range.
*   Exception to this case is for the plots for ISPJ and ISTJ, but this maybe due to the fact that there are significantly less no. of posts available for these personality types (further shown by the bar plots below)
*   We can see that there is no correlation observed between variance of word count and the words per comment.
*   But there is a weak negative correlation observed between the 2 features for few personalities. Maybe this could be due to the low no. of posts available for that type in the given Kaggle dataset.
*   No useful inferences can be made by analyzing the individual jointplots as the total no of posts for each personlaity type is different.

Hence, these features will not be useful in building our Personality prediction model.

### DISTANCE PLOT

This seaborn visualization method shows the histogram distribution of data for a single column.

In [ ]:
df["length_posts"] = df["posts"].apply(len)
sns.distplot(df["length_posts"]).set_title("Distribution of Lengths of all 50 Posts")

We can see that most no of lengthly posts have between 7000-9000 words.
The line that you see represents the kernel density estimation. It is a fundamental data smoothing problem where inferences about the population, based on a finite data sample. This kernel density estimate is a function defined as the sum of a kernel function on every data point.

In [ ]:
#Finding the most common words in all posts.
words = list(df["posts"].apply(lambda x: x.split()))
words = [x for y in words for x in y]
Counter(words).most_common(40)

*   The posts majorly contain general words like : I, I'm, so, me, or, if, and, can etc. It is safe to assume that these words won't really provide any useful information to train the ML model as most of them are stop-words, stem-words, or other useless words.
*   Hence quite a lot pre-processing is required for individual user posts for each peronality type in the given MBTI dataset